# 코스피50 주가 예측
데이터 출처:
http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201010106

## (1) 데이터

In [ ]:
!pip install finance-datareader

In [ ]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
code_list = pd.read_csv('/content/drive/MyDrive/데이터/주가예측/data_0550_20221210.csv', encoding='cp949')

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
def make_code(x):
    x = str(x)
    return '0'*(6-len(x)) + x

In [ ]:
code_list['종목코드'] = code_list['종목코드'].apply(make_code)

In [ ]:
code_list = code_list['종목코드']

### 데이터 만들기(train, val)

In [ ]:
merge_stock_list =[]
for code in code_list: 
    stock_data = fdr.DataReader(code, '20191201','20221130').reset_index()
    stock_data[['Open1', 'High1', 'Low1', 'Close1', 'Volume1', 'Change1']] = stock_data[['Open', 'High', 'Low', 'Close', 'Volume', 'Change']].shift(1)
    stock_data[['Open3', 'High3', 'Low3', 'Close3', 'Volume3', 'Change3']] = stock_data[['Open', 'High', 'Low', 'Close', 'Volume', 'Change']].shift(3)
    stock_data[['Open7', 'High7', 'Low7', 'Close7', 'Volume7', 'Change7']] = stock_data[['Open', 'High', 'Low', 'Close', 'Volume', 'Change']].shift(7)
    stock_list = stock_data.values.tolist()

    for row in stock_list:                   # 불러온 주가 데이터를 1줄씩 불러옴
        row.append(code)                     # 주가 데이터에 기업 코드를 추가    
        merge_stock_list.append(row)

In [ ]:
train2 = pd.DataFrame(merge_stock_list,  columns = ['date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change',
                                                    'Open1', 'High1', 'Low1', 'Close1', 'Volume1', 'Change1',
                                                    'Open3', 'High3', 'Low3', 'Close3', 'Volume3', 'Change3', 
                                                    'Open7', 'High7', 'Low7', 'Close7', 'Volume7', 'Change7', 'code'] )

In [ ]:
df = train2[train2['date'] >= '2020-01-01']

In [ ]:
df.dropna(inplace = True, axis = 0)

In [ ]:
df.reset_index(inplace = True, drop = True)

In [ ]:
df.isna().sum().sum()

0

In [ ]:
df.shape

(33271, 26)

### 파생변수

y 상승 1, 하락 0 예측

In [ ]:
df['increase'] = np.where(df['Change'] > 0, 1, 0) 

In [ ]:
df['increase'].value_counts()

0    17834
1    15437
Name: increase, dtype: int64

시가 종가 차이

In [ ]:
df['diff_se1'] = df['Open1'] - df['Close1']
df['diff_se3'] = df['Open3'] - df['Close3']
df['diff_se7'] = df['Open7'] - df['Close7']

고가 저가 차이

In [ ]:
df['diff_lh1'] = df['Low1'] - df['High1']
df['diff_lh3'] = df['Low3'] - df['High3']
df['diff_lh7'] = df['Low7'] - df['High7']

1, 3, 7일의 종가 변화

In [ ]:
close_array = []
for i in range(len(df)):
  x = [1, 3, 7]
  y = [df.iloc[i,10],df.iloc[i,16], df.iloc[i,22]]
  fit_line = np.polyfit(x, y, 1)
  close_array.append(fit_line[0])

In [ ]:
df['close_variation'] = close_array

1, 3, 7일의 규모 변화

In [ ]:
volume_array = []
for i in range(len(df)):
  x = [1, 3, 7]
  y = [df.iloc[i,11],df.iloc[i,17], df.iloc[i,23]]
  fit_line = np.polyfit(x, y, 1)
  volume_array.append(fit_line[0])

In [ ]:
df['volume_variation'] = volume_array

전날 코스피 지수

In [ ]:
kospi = fdr.DataReader('KS11', '2019-12-01').reset_index()
kospi[['kospi_close1']] = kospi[['Close']].shift(1)

kospi.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Close'], inplace = True, axis= 1)
kospi.columns = ['date', 'kospi_close1']

In [ ]:
df = pd.merge(df, kospi,on='date', how= 'left' )

달러당 원화 환율

In [ ]:
import datetime
df['date'] = pd.to_datetime(df['date'])

In [ ]:
dollar = fdr.DataReader('USD/KRW', '2019-12-01').reset_index()
dollar[['USD/KRW1']] = dollar[['Close']].shift(1)

dollar.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Close'], inplace = True, axis= 1)
dollar.columns = ['date', 'USD/KRW1']

In [ ]:
df = pd.merge(df, dollar ,on='date', how= 'left' )

엔화 원화 환율

In [ ]:
jp = fdr.DataReader('JPY/KRW', '2019-12-01').reset_index()
jp[['JPY/KRW1']] = jp[['Close']].shift(1)

jp.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Close'], inplace = True, axis= 1)
jp.columns = ['date', 'JPY/KRW1']

In [ ]:
df = pd.merge(df, jp ,on='date', how= 'left' )

비트코인 원화 가격

In [ ]:
coin = fdr.DataReader('BTC/KRW', '2019-12-01').reset_index()
coin[['BTC/KRW1']] = coin[['Close']].shift(1)

coin.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Close'], inplace = True, axis= 1)
coin.columns = ['date','BTC/KRW1']

In [ ]:
df = pd.merge(df, coin ,on='date', how= 'left' )

### 결측치

In [ ]:
df.dropna(axis = 0, inplace = True)
df.reset_index(inplace = True, drop = True)

In [ ]:
df.isna().sum().sum()

0

### 변수선택

In [ ]:
df = df[['code','date', 'increase',
      'Open1', 'High1', 'Low1', 'Close1', 'Volume1', 'Change1', 
      'Open3', 'High3','Low3', 'Close3', 'Volume3', 'Change3',
      'Open7', 'High7', 'Low7','Close7', 'Volume7', 'Change7', 
      'diff_se1', 'diff_se3', 'diff_se7', 'diff_lh1', 'diff_lh3', 'diff_lh7',
      'close_variation', 'volume_variation', 
      'kospi_close1', 'USD/KRW1', 'JPY/KRW1', 'BTC/KRW1']]